In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
    
import h5py
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [3]:
try:
    import dill as pickle
except ImportError:
    import pickle

In [4]:
train_features = load(open("encoded_train_images.pickle", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=52


In [5]:
train_descriptions = load(open("train_descriptions.pickle", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=52


In [6]:
embedding_matrix = load(open("Embedding_Matrix.pkl", "rb"))
print('Embedding Matrix shape = (%d, %d)' % (embedding_matrix.shape))

Embedding Matrix shape = (4, 200)


In [7]:
wordtoix = load(open("wordtoix.pkl", "rb"))
print('Wordtoix Length = %d' % len(wordtoix))

Wordtoix Length = 3


In [8]:
max_length = 34 # computer from file 2
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [9]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    print("uslo")
    X1, X2, y = list(), list(), list()
    n=0
    while 1:
        for key, desc_list in descriptions.items():

            n+=1
            #print(photos)
            try:
                photo = photos["usicGenerator/textImages/"+key+".png"]
            except:
                photo = photos["usicGenerator/textImages/"+key+".png "]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
#                 yield [[array(X1), array(X2)], array(y)]
#                 for i in range(n):
#                     newArr.append((X1))
                X1, X2, y = list(), list(), list()
                n=0
    print("k")
    return X1, X2, y

In [10]:
max_length = 35 # computer from file 2
epochs = 10
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [11]:
train_features['usicGenerator/textImages/10-Yard Fight - square7.png'].shape

(2048,)

In [12]:
max_length = 35 
vocab_size = embedding_matrix.shape[0]
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath
embedding_dim = embedding_matrix.shape[1]
max_length,vocab_size ,embedding_dim

(35, 4, 200)

In [13]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
my_model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [56]:
pip install numpy==1.19.5 --user

^C
Note: you may need to restart the kernel to use updated packages.


In [14]:
my_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 35)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 35, 200)      800         input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
_______________________________________________________________________________________

In [15]:
my_model.layers[2].set_weights([embedding_matrix])
my_model.layers[2].trainable = False

In [16]:
my_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001))

In [17]:
my_model.save('.model_N' + 'test' + '.h5')

In [18]:
epochs = 20
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [19]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    my_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    my_model.save('model_NN' + str(i) + '.h5')

uslo


KeyboardInterrupt: 

In [50]:
with open('Caption_generator_Model.pkl', 'wb') as model:
    pickle.dump(my_model, model)

TypeError: cannot pickle '_thread.RLock' object